In [ ]:
%%capture
!pip install neptune-client psutil
!git clone https://github.com/Cho-D-YoungRae/URP_PD.git
%cd URP_PD

In [ ]:
import dataset
import object_detection
from utils import *
import eval

import torch
from torch.utils.data import Dataset, DataLoader

import os
import json
import numpy as np
import argparse
from tqdm.auto import tqdm
import time
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using <{device}> device")

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
Using <cuda> device


In [ ]:
# ====== constants ======#
label_map = {'background': 0, 'person': 1}
rev_label_map = {v: k for k, v in label_map.items()} 

## setting

In [ ]:
from torch.backends import cudnn
import random

cudnn.benchmark = True

# ====== Random Seed Initialization ====== #
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.baselineID = 23

# ====== Dataset ====== #
args.ch_option = {'num_ch': 1,
                  'img_type': 'lwir',
                  'one_ch_option': 'mean'}
args.val_split = 0.1

# ====== Model ====== #
args.base_model = 'VGG16bnBase'
args.n_classes = len(label_map)
args.is_sds = True
args.usages_seg_feats = [True, False, False, False, False, False]


# ====== Optimizer & Training ====== #
args.optim = 'Adam'
args.lr = 5e-4
args.twice_b_lr = True
args.weight_decay = 5e-4

args.epochs = 100
args.train_batch_size = 32
args.test_batch_size = 64

args.decay_lr_at = [int(args.epochs/6)*4,
                    int(args.epochs/6)*5]
args.decay_lr_to = 0.1

## neptune init

In [ ]:
import neptune.new as neptune

api_token = 
run = neptune.init(
    project='jodyr/urp',
    source_files=['*.py'],
    api_token=api_token,
    # run='PD-25',
    )

run["parameters"] = vars(args)


https://app.neptune.ai/jodyr/urp/e/PD-62
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## train

In [ ]:
def train(train_loader, model, criterion, optimizer):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: MultiBox loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    print_freq = len(train_loader) // 4

    start = time.time()

    # Batches
    for i, (images, bboxes, category_ids, is_crowds, seg_labels) in enumerate(train_loader):
        data_time.update(time.time() - start)

        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        bboxes = [b.to(device) for b in bboxes]
        category_ids = [c.to(device) for c in category_ids]
        seg_labels = seg_labels.to(device)

        # Forward prop.
        predicted_locs, predicted_scores, pred_segs = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        # Loss
        loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids,
                         pred_segs, seg_labels)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print(f'[{i}/{len(train_loader)}]\t'
                  f'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  f'Loss {losses.val:.4f} ({losses.avg:.4f})\t')
    del predicted_locs, predicted_scores, images, bboxes, category_ids

    train_loss = losses.avg
    return train_loss        

## validation

In [ ]:
# def validation(val_loader, model, criterion):
#     model.eval()

#     num_batches = len(val_loader)
#     losses = AverageMeter()
#     with torch.no_grad():
#         for i, (images, bboxes, category_ids, _) in enumerate(val_loader):
#             images = images.to(device)
#             bboxes = [b.to(device) for b in bboxes]
#             category_ids = [l.to(device) for l in category_ids]

#             predicted_locs, predicted_scores = model(images)
#             loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids).item()

#             losses.update(loss, images.size(0))

#     val_loss = losses.avg
#     return val_loss

## checkpoint

In [ ]:
checkpoint = os.path.join('/content/drive/MyDrive/2021.summer_URP/PD/checkpoint',
                          str(args.baselineID)+'.pth.tar')
checkpoint = checkpoint if os.path.isfile(checkpoint) else None
print(f"checkpoint: {checkpoint}")

checkpoint: None


In [ ]:
if checkpoint is None:
    start_epoch = 1
    lr = args.lr
    model = object_detection.SDSSSD300(n_classes=args.n_classes,
                                       base=args.base_model,
                                       ch_option=args.ch_option,
                                       usages_seg_feats=args.usages_seg_feats)
    if args.twice_b_lr:
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        optimizer = getattr(torch.optim, args.optim)(params=[{'params': biases, 'lr': 2 * lr}, 
                                                            {'params': not_biases}],
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)
    else:
        optimizer = getattr(torch.optim, args.optim)(params=model.parameters(),
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']


model = model.to(device)
criterion = object_detection.SDSMultiBoxLoss(priors_cxcy=model.priors_cxcy,
                                             usages_seg_feats=model.usages_seg_feats).to(device)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth




Loaded base model.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


## dataset init

In [ ]:
workers = 4
train_dataset = dataset.KaistPDDataset(ch_option=args.ch_option,
                                       is_sds=args.is_sds)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=args.train_batch_size, 
                                           shuffle=True,
                                           collate_fn=dataset.sds_collate_fn,
                                           num_workers=workers,
                                           pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## experiment

In [11]:
checkpoint_dir = '/content/drive/MyDrive/2021.summer_URP/PD/checkpoint'
checkpoint_path = os.path.join(checkpoint_dir,
                               str(args.baselineID)+'.pth.tar')

epochs = args.epochs
decay_lr_at = args.decay_lr_at
save_freq = 5
eval_freq = 10


# Epochs
for epoch in range(start_epoch, epochs+1):
    print(f"# ====== Epoch {epoch} ====== # {datetime.now()}")
    # Decay learning rate at particular epochs
    if epoch in decay_lr_at:
        adjust_learning_rate(optimizer, args.decay_lr_to)

    # One epoch's training
    train_loss = train(train_loader=train_loader,
                       model=model,
                       criterion=criterion,
                       optimizer=optimizer)
    run['train/loss'].log(train_loss)
    if epoch % save_freq == 0:
        save_checkpoint(epoch, model, optimizer, checkpoint_path)

    if epoch % eval_freq == 0:
        eval.evaluate(model, ch_option=args.ch_option)

# ====== Epoch 1 ====== # 2021-08-02 02:53:36.205351


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[0/273]	Batch Time 82.317 (82.317)	Data Time 76.499 (76.499)	Loss 12.5100 (12.5100)	
[68/273]	Batch Time 13.092 (5.439)	Data Time 12.546 (4.818)	Loss 4.0839 (5.1831)	
[136/273]	Batch Time 11.411 (4.889)	Data Time 10.863 (4.305)	Loss 3.9322 (4.5941)	
[204/273]	Batch Time 14.685 (4.700)	Data Time 14.144 (4.129)	Loss 4.0224 (4.3570)	
[272/273]	Batch Time 15.559 (4.581)	Data Time 11.285 (4.002)	Loss 3.5948 (4.2036)	
# ====== Epoch 2 ====== # 2021-08-02 03:14:26.995484
[0/273]	Batch Time 4.301 (4.301)	Data Time 3.666 (3.666)	Loss 3.4804 (3.4804)	
[68/273]	Batch Time 2.518 (1.033)	Data Time 1.961 (0.403)	Loss 3.3741 (3.5641)	
[136/273]	Batch Time 2.379 (0.993)	Data Time 1.719 (0.364)	Loss 3.3236 (3.4841)	
[204/273]	Batch Time 1.361 (0.980)	Data Time 0.706 (0.350)	Loss 3.2895 (3.4408)	
[272/273]	Batch Time 0.469 (0.970)	Data Time 0.000 (0.342)	Loss 3.2402 (3.4029)	
# ====== Epoch 3 ====== # 2021-08-02 03:18:51.831961
[0/273]	Batch Time 4.472 (4.472)	Data Time 3.811 (3.811)	Loss 3.2993 (3.2993

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.26s).
Accumulating evaluation results...
DONE (t=0.10s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 33.21%
Recall: 0.7800436999271668
# ====== Epoch 11 ====== # 2021-08-02 04:05:54.372515


...


Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.32s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 26.20%
Recall: 0.8299078667611623
# ====== Epoch 91 ====== # 2021-08-02 10:12:16.120056


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0/273]	Batch Time 4.339 (4.339)	Data Time 3.627 (3.627)	Loss 2.5241 (2.5241)	
[68/273]	Batch Time 1.878 (1.013)	Data Time 1.161 (0.376)	Loss 2.3661 (2.1171)	
[136/273]	Batch Time 0.632 (1.001)	Data Time 0.001 (0.363)	Loss 1.7508 (2.1005)	
[204/273]	Batch Time 1.622 (1.000)	Data Time 0.993 (0.364)	Loss 2.2576 (2.1211)	
[272/273]	Batch Time 0.471 (0.986)	Data Time 0.000 (0.351)	Loss 2.0845 (2.1181)	
# ====== Epoch 92 ====== # 2021-08-02 10:16:45.449195
[0/273]	Batch Time 4.809 (4.809)	Data Time 4.164 (4.164)	Loss 2.0389 (2.0389)	
[68/273]	Batch Time 1.741 (1.031)	Data Time 1.139 (0.397)	Loss 2.3106 (2.1126)	
[136/273]	Batch Time 2.244 (1.006)	Data Time 1.630 (0.371)	Loss 2.0456 (2.1068)	
[204/273]	Batch Time 1.507 (1.008)	Data Time 0.880 (0.372)	Loss 2.3681 (2.0933)	
[272/273]	Batch Time 0.475 (0.993)	Data Time 0.000 (0.359)	Loss 2.2379 (2.0910)	
# ====== Epoch 93 ====== # 2021-08-02 10:21:16.693807
[0/273]	Batch Time 4.665 (4.665)	Data Time 4.074 (4.074)	Loss 2.0851 (2.0851)	
[68/273]	

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.31s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 26.44%
Recall: 0.8226950354609929
